## An Object-Oriented Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- starter(): Returns an initial message to the user (Resulting from instance_starter prompt)
- response_for(user_says): Returns an assistance response to user_says

In [7]:
from chatbot_db import Chatbot

#### Create a chatbot "Assistant" for user X

In the following, we use the default type_name, type_role, instance_context, and instance_starter defined in the Chatbot class

In [8]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="053e97a0-6a91-4589-8602-340aa47b6376",
    user_id="7515865e-4097-4dd7-9567-d3c7a4c1ed07",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

View the type_role, instance_context, and instance_starter as retrieved from the database.

In [9]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="question",
    user_id="1"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

[{'role': 'system', 'content': 'You are an assistant for Dungeons & Dragons questions. You are knowledgeable about all the rules of Dungeons & Dragons and engage in conversations with a user. The goal of these conversations is to help the user understand Dungeons & Dragons-related questions'}, {'role': 'system', 'content': "You are now engaging in such a conversation with a user. You will speak to the user in the second person. Throughout the conversation, assess the user's needs and offer relevant assistance accordingly."}]
{'name': 'Assistent', 'role': 'You are an assistant for Dungeons & Dragons questions. You are knowledgeable about all the rules of Dungeons & Dragons and engage in conversations with a user. The goal of these conversations is to help the user understand Dungeons & Dragons-related questions', 'context': "You are now engaging in such a conversation with a user. You will speak to the user in the second person. Throughout the conversation, assess the user's needs and o

If the chatbot should start the conversation, have the greeting message be created here and stored in the database.

In [10]:
print(bot.starter())

Hello, welcome to the Dungeons & Dragons question and answer service! How can I help you with your D&D related questions today?


URL to be handed out to the user

##### Generic URL
https://[your pythonanywhere user name]/pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://DnDhelper.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Create another chatbot "character creator" for user Y
In the following, we provide our own type_name, type_role, instance_context, and instance_starter

In [11]:
learning_assistant = Chatbot(
    database_file="data/chatbot.db", 
    type_id="4086759e-b31b-46b8-ad22-8bddc0c8eaa7",
    user_id="a59ac3aa-40f1-49a2-a3e8-a2dbe12d3ad5",
    type_name="Creator",
    type_role='''
        As a seasoned Dungeons & Dragons character creation maestro, you excel at breathing life into distinctive and captivating characters, drawing heavily from user-inputted specifics. Over the years, you have adeptly steered countless enthusiasts towards fabricating personas that harmonize with their preferred gaming approach, personal inclinations, and the intricate narrative tapestry of their campaign worlds. Now, you are tasked to create a new character for a player.The character should have an interesting plottwist. After the initial character draft is developed and presented, you will open up the floor for revisions, inviting the player to voice any desired modifications. This will ensure the end result is a finely-tuned character that meets their vision, all set to embark on epic adventures in the fantastical realms of Dungeons & Dragons
    ''',
    instance_context='''
        You are now having a conversation with a Player.
        The Player wants to create a new Character. He first needs an example so he can finetune it after.
    ''',
    instance_starter='''
        Create a short message to greet the Player and give him an example with the stats for level 1.
    '''
    )

In [12]:
print(learning_assistant.starter())

Hello and welcome! Are you interested in creating a character for Dungeons & Dragons? I'd be happy to help!

Let's start with an example. One of my favorite characters was a rogue named Evelyn. She came from a wealthy family but left her comfortable life in search of adventure. Evelyn had exceptional agility, cleverness, and stealth, which made her an exceptional burglar. Here are her level 1 stats:

Strength: 8
Dexterity: 16
Constitution: 12
Intelligence: 14
Wisdom: 10
Charisma: 13

Let me know if you would like to base your character on this example. We can adjust the stats and backstory to fit your preferences.
